### 공통 설정: 모듈 import, model, API KEY 등 설정

In [ ]:
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
import os
from dotenv import load_dotenv

# .env 파일에서 환경 변수 로드
load_dotenv()

# ChatOpenAI 모델을 Groq API 엔드포인트로 설정
llm = ChatOpenAI(
    base_url="https://api.groq.com/openai/v1",
    model="meta-llama/llama-4-scout-17b-16e-instruct",
    temperature=0.7 
)

# OpenAI API 키 (Groq API 키로 사용) 설정
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")

### 문제 1-1 : 기본 체인 만들기 - AI 요리사

In [ ]:
# 1. PromptTemplate을 사용하여 프롬프트 작성
template = """
당신은 주어진 재료로 만들 수 있는 요리를 추천하고 간단한 레시피를 제공하는 AI 요리사입니다.

사용자가 입력한 재료: {ingredients}

이 재료들로 만들 수 있는 요리를 추천하고, 간단한 레시피를 한국어로 제공해주세요.
추천 요리: [요리 이름]
재료: [사용된 재료]
조리법:
1. ...
2. ...
"""
prompt = PromptTemplate.from_template(template)

# 2. StrOutputParser로 결과를 문자열로 출력
output_parser = StrOutputParser()

# 3. LCEL(|) 문법을 사용하여 체인 연결
chain = prompt | llm | output_parser

# 4. 사용자 입력 받기 및 결과 출력
def recommend_dish(ingredients_input):
    print(f"입력: \"{ingredients_input}\"")
    response = chain.invoke({"ingredients": ingredients_input})
    print(f"\n{ingredients_input}로 만들 수 있는 요리를 추천드립니다!\n")
    print(response)

print("AI 요리사를 시작합니다. 재료를 입력해주세요 (예: 토마토, 양파, 치즈).")
user_ingredients = input("재료 입력: ")
recommend_dish(user_ingredients)

### 문제 1-2 : 2단계 체인 만들기 - 영화 추천 시스템 

In [ ]:
from langchain_core.prompts import ChatPromptTemplate

print("--- 2단계 영화 추천 시스템을 시작합니다 ---")

# 1단계 체인: 장르를 입력받아 영화 1편 추천
# ChatPromptTemplate 사용: "system"과 "human" 메시지 지정
first_stage_prompt = ChatPromptTemplate.from_messages([
    ("system", "당신은 사용자에게 영화 장르에 맞는 영화 한 편을 추천해주는 친절한 영화 큐레이터입니다."),
    ("human", "{genre} 장르의 영화 한 편을 추천해 주세요. 영화 제목만 알려주세요.")
])

# 1단계 체인 정의
first_stage_chain = first_stage_prompt | llm | StrOutputParser()

# 2단계 체인: 추천받은 영화의 3줄 줄거리 요약 제공
# ChatPromptTemplate 사용: "system"과 "human" 메시지 지정
second_stage_prompt = ChatPromptTemplate.from_messages([
    ("system", "당신은 주어진 영화의 줄거리를 3문장으로 간결하게 요약해주는 전문가입니다."),
    ("human", "다음 영화의 줄거리를 3문장으로 요약해주세요: {movie_title}")
])

# 2단계 체인 정의
second_stage_chain = second_stage_prompt | llm | StrOutputParser()

# 두 체인을 LCEL로 연결하여 전체 과정 구현
def recommend_and_summarize_movie(genre_input):
    print(f"\n입력 장르: \"{genre_input}\"")

    # 1단계 실행: 영화 추천
    print("\n--- 1단계 실행: 영화 추천 ---")
    recommended_movie = first_stage_chain.invoke({"genre": genre_input})
    print(f"1단계 결과: \"{recommended_movie.strip()}\" 추천")

    # 2단계 실행: 줄거리 요약
    print("\n--- 2단계 실행: 줄거리 요약 ---")
    summary = second_stage_chain.invoke({"movie_title": recommended_movie.strip()}) # strip()으로 불필요한 공백 제거
    print(f"영화: {recommended_movie.strip()}\n줄거리:\n{summary.strip()}")

# 사용자 입력 받기 및 실행
user_genre = input("\n추천받고 싶은 영화 장르를 입력하세요 (예: 액션, 로맨스, 공포): ")
recommend_and_summarize_movie(user_genre)

### 문제 1-3 : FewShotPromptTemplate과 시스템 메시지 활용

In [ ]:
from langchain_core.prompts import (
    FewShotChatMessagePromptTemplate,
    ChatPromptTemplate
)

print("\n--- 뉴스 키워드 추출기를 시작합니다 ---")

# 1. 예시 프롬프트 (대화형)
# 각 예시의 형식(human과 ai 메시지)을 정의합니다.
example_prompt = ChatPromptTemplate.from_messages([
    ("human", "{news}"),
    ("ai", "키워드: {keywords}")
])

# 2. 예시 데이터 (Few-shot examples)
# 뉴스 텍스트와 그에 해당하는 핵심 키워드 3개를 포함하는 예시를 정의합니다.
# 최소 3개의 예시를 포함해야 합니다.
examples = [
    {
        "news": "삼성전자가 내년 초에 자체적으로 개발한 인공지능(AI) 가속기를 처음으로 출시할 예정이다. 이는 AI 반도체 시장에서 지배적인 위치를 차지하고 있는 엔비디아의 독점을 도전하고, 세계 최고의 반도체 제조업체로서의 지위를 다시 확립하려는 삼성전자의 노력으로 해석된다.",
        "keywords": "삼성전자, 인공지능, 엔비디아"
    },
    {
        "news": "세계보건기구(WHO)는 최근 새로운 건강 위기에 대응하기 위해 국제 협력의 중요성을 강조했다. 전염병 대응 역량의 강화와 글로벌 보건 시스템의 개선이 필요하다고 발표했다.",
        "keywords": "세계보건기구, 건강위기, 국제협력"
    },
    {
        "news": "테슬라가 새로운 전기차 모델 '사이버트럭'을 공개하며 자동차 시장에 큰 파장을 일으켰다. 독특한 디자인과 강력한 성능으로 소비자들의 이목을 집중시키고 있다.",
        "keywords": "테슬라, 전기차, 사이버트럭"
    },
    {
        "news": "국제우주정거장(ISS)에서 새로운 생물학 실험이 시작되었다. 미세 중력 환경이 식물 성장에 미치는 영향을 연구하여 우주 농업 기술 발전에 기여할 예정이다.",
        "keywords": "국제우주정거장, 생물학실험, 우주농업"
    }
]

# 3. Few-Shot 프롬프트
# 위에서 정의한 예시 프롬프트와 예시 데이터들을 Few-Shot 프롬프트로 묶습니다.
few_shot_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
)

# 4. 최종 프롬프트
# 시스템 메시지와 Few-Shot 프롬프트, 그리고 사용자 입력 프롬프트를 결합합니다.
# 시스템 메시지는 LLM에게 역할을 명확히 지시합니다.
final_prompt = ChatPromptTemplate.from_messages([
    ("system", "당신은 주어진 뉴스 기사에서 핵심 키워드 3개를 추출하는 전문가입니다. '키워드: [키워드1], [키워드2], [키워드3]' 형식으로 추출하세요."),
    few_shot_prompt, # Few-Shot 예시들이 여기에 삽입됩니다.
    ("human", "{input}") # 사용자가 실제로 키워드를 추출할 뉴스 텍스트가 들어갈 자리입니다.
])

# 5. 체인 연결
# 최종 프롬프트, LLM, 그리고 StrOutputParser를 LCEL로 연결합니다.
keyword_extraction_chain = final_prompt | llm | StrOutputParser()

# 6. 테스트 뉴스
test_news = "제미나이 2.0 플래시는 현재 구글 AI 스튜디오(Google AI Studio) 및 버텍스 AI(Vertex AI)에서 제미나이 API를 통해 개발자에게 실험 모델로 제공됩니다. 모든 개발자는 멀티모달 입력 및 텍스트 출력을 사용할 수 있으며, 텍스트 음성 변환(text-to-speech) 및 네이티브 이미지 생성은 일부 파트너들을 대상으로 제공됩니다. 내년 1월에는 더 많은 모델 사이즈와 함께 일반에 공개될 예정입니다."

# 7. 실행 및 결과 출력
print(f"\n입력 뉴스:\n{test_news}")
print("\n--- 키워드 추출 결과 ---")
extracted_keywords = keyword_extraction_chain.invoke({"input": test_news})
print(extracted_keywords)

print("\n--- 추가 테스트 ---")
another_news = "윤석열 대통령은 오늘 용산 대통령실에서 한-아프리카 정상회의에 참석하기 위해 방한한 아프리카 정상들과 만찬을 가졌다. 이번 만찬은 한-아프리카 관계의 새로운 지평을 열고 상호 협력을 강화하는 중요한 자리였다."
print(f"\n입력 뉴스:\n{another_news}")
extracted_keywords_2 = keyword_extraction_chain.invoke({"input": another_news})
print(extracted_keywords_2)